In [1]:
import torch
from torch.utils.data import DataLoader
from glob import glob
import os
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

from components.mymodel import load_model, get_model
from components.helper import train

In [2]:
from torch.utils.data import Dataset
from torchvision import io

class SoilDataset_phone(Dataset):
    def __init__(self, dataset_path:str, image_set:str, transform=None):
        assert os.path.exists(dataset_path), f"Path is not exist."

        image_folder = os.path.join(dataset_path, image_set)
        assert os.path.exists(image_folder), f"image_set={image_set} is not exists in {dataset_path}."
        self.imgs = glob(os.path.join(image_folder,'*'))
        print(f"Found {len(self.imgs)} images in {image_folder}.")
        
        self.transform = transform

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        img_path = self.imgs[idx]
        path, img_name_w_ext = os.path.split(img_path)
        name, ext = os.path.splitext(img_name_w_ext)
        # print(name)
        id, phone_brand, OM, lab_no, soil_character, place, variant = name.split('_')
        y = float(OM)
        y = torch.tensor(y)
        # print(y)
        X = io.read_image(img_path)
        if self.transform:
            X = self.transform(X)
        return X.float(), y.float(), img_name_w_ext

In [3]:
#We can check whether we have gpu
DEVICE = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
# DEVICE = 'cpu'
print("Device: ", DEVICE)

Device:  cuda:0


In [4]:
def train_model(model, image_set:str, epochs:int, lr:float, save_path:str):
    preprocess = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(224),
        transforms.CenterCrop(224),
        # transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
        # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])


    dataset = SoilDataset_phone(dataset_path='./dataset/phones/', image_set=image_set, transform=preprocess)
    loader = DataLoader(dataset=dataset, batch_size=100, shuffle=True, num_workers=16)
    model, train_losses = train(model, loader, epochs, lr, DEVICE, save_path)
    plt.plot(train_losses)
    plt.title(image_set)
    plt.show()
    return model, train_losses

In [6]:
# image_set_list = next(os.walk('./dataset/phones/'))[1]
train_losses = []
# mobilenet_v3_large
# resnet50
# efficientnet_v2_l
# alexnet
model_name = 'alexnet'
weight_path = f'./weight/{model_name}'
epochs = 300
lr = 0.00001

if(os.path.exists(weight_path) == False):
    os.makedirs(weight_path)
# for image_set in image_set_list:
    # if(image_set == 'Apple'): continue
image_set = 'All'
save_path =  os.path.join(weight_path,f"{image_set}.pth")
# Continue
# model = get_model(model_name=model_name, image_set=image_set)
# From scratch
model = load_model(model_name=model_name)
model, train_loss = train_model(model, image_set, epochs, lr, save_path=save_path)
train_losses.append(train_loss)
# torch.save(model.state_dict(), os.path.join(weight_path,f"{image_set}.pth"))

Found 2355 images in ./dataset/phones/All.
90.59462571144104 0 tensor(287.9718)
99.48640298843384 1 tensor(181.9653)
99.16231203079224 2 tensor(161.5345)
98.94897222518921 3 tensor(152.2581)
99.16668319702148 4 tensor(146.9093)
99.69530010223389 5 tensor(139.9800)
96.74688911437988 6 tensor(130.2754)
98.65414619445801 7 tensor(112.4331)
99.21304821968079 8 tensor(93.1612)
97.01628112792969 9 tensor(77.7088)
98.39907264709473 10 tensor(69.6770)
99.22574925422668 11 tensor(65.7657)
98.34354782104492 12 tensor(64.0383)
96.12553906440735 13 tensor(61.3674)
98.1189432144165 14 tensor(59.2101)
108.18712401390076 15 tensor(58.0132)
97.08176112174988 16 tensor(55.7555)
101.9362223148346 17 tensor(55.1697)
98.91322755813599 18 tensor(53.9192)
99.35451078414917 19 tensor(52.5128)
99.33327913284302 20 tensor(58.2154)
98.50295948982239 21 tensor(52.5908)
98.2433910369873 22 tensor(51.5805)
98.2453784942627 23 tensor(52.9168)
98.09355545043945 24 tensor(50.2367)
99.99948239326477 25 tensor(50.3010)

KeyboardInterrupt: 